<a href="https://colab.research.google.com/github/mlfa03/MLOPs/blob/main/Helm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deploying Kubernetes with Helm

Helm is called the package manager for Kubernetes. It makes it easier to package and deploy software on a Kubernetes cluster using app definitions called charts.

A chart is a package that can be shared and reused, which contains an application’s Kubernetes resource definitions (YAML files) and some templating logic.

Helm keeps track of the charts it installs on your Kubernetes cluster by adding labels to the objects created. These labels look like app.kubernetes.io/managed-by=Helm and app.kubernetes.io/instance: myapp.

**Helm concepts**

* Chart is a package in Helm. It has a name, and contains a set of Kubernetes resource definitions that are used to deploy your application.

* Repository is an online collection of charts. It has a URL. You can search, download and install charts from a repository.

* Release is an instance or a deployment of a chart. When you perform a helm install command, you are creating a new release of that chart on your Kubernetes cluste

### Finding and viewing charts

**Repositories** often contain many charts, and can be found on an open source repository server like Chartmuseum, or a public chart repository like ArtifactHub. 


In [ ]:
helm repo add [name] [url]                              #Add a remote chart repository
helm repo list                                          #list all your repositories
helm search hub prometheus                              #search for 'prometheus' chart on Artifact hub
helm search repo prometheus                             #search for 'prometheus' chart on all repositories configured on your system 

### Installing charts and managing charts

The most common way people make use of Helm is to find and install charts onto a Kubernetes cluster. Here are the main commands you might use when installing charts.

In [ ]:
#View information about a chart:

helm repo add concourse https://concourse-charts.storage.googleapis.com/
helm show chart concourse/concourse
helm show values concourse/concourse

#If you have mentioned a dependency in your Chart.yaml, and you need to download it:
helm dependency update

#To simulate installing a chart without actually installing it, do a dry run:
helm install --dry-run ...

#This will install a chart from a repository, so you should ensure that you’ve set up the repository first.
helm install myapp-instance sourcerepo/myapp

#This allows you to specify the URL to the repository when you install (it initializes the repository):
helm install --repo http://charts.example.com my-app-instance myapp

#Install a chart from a packaged file
helm install myapp-instance ./path/to/myappchart.tgz

#Install a chart from an unpacked chart directory
helm install myapp-instance ./path/to/chart

#Install a template with a values override file
helm install -f ./my-extra-values.yml ./mychart

#When you want to install a Helm chart, and override its values explicitly at the command line:
helm install --set key1=val1,key2=val2 [name] [chart]



### Getting information about deployed charts

In [ ]:
helm list                                             #When you want to see all the charts that have been deployed on your cluster (releases)
helm get manifest <release-name>                      #When you want to get the YAML of all of the objects that comprise a release, or see all of the objects that were created when you did a helm install
helm status <release-name>                            #When a chart is deployed (a release is created), a status message is shown. To print this message again
helm uninstall <release-name>                         #uninstalling a release

### Deploying your own charts

Packaging your own software for use with Helm is known as creating a chart. You can create a chart yourself from scratch, or you can use the helm create chart which will initialise a chart for you.

In [ ]:
#This will create a new skeleton or boilerplate chart at ./mychart, which is ready for you to edit and customise.
helm create [path]
helm create mychart

#Use helm template if you want Helm to show the output YAML from your Helm chart, if you were to helm install it. This is useful when debugging your chart:
helm template [NAME] [CHART]
helm template my-app-instance sourcerepo/my-app
helm template my-app-instance .



### Example

In [ ]:
#Installing kube-metrics chart:

kubectl create ns metrics                                                       #create a namespace called metrics to install the chart
helm install kube-state-metrics bitnami/kube-state-metrics -n metrics           #
helm ls -n metrics 

#Inspect the chart installed:
kubectl get all -n metrics

#Checking the logs for the pod created:
kubectl logs [name-of-pod] -n metrics

kubeclt port-forward svc/kube-state-metrics 8080:8080 -n metrics

### Directories and files created by helm

**Chart**

* The chart.yaml file contains high level information about your chart that can be passed into the helm templating engine

**Values**
* Here, we see lots of key value pairs that allow us to configure the container or containers running in a pod. Some of these fields include the image repository so that the pod can pull the correct image for the container, a place to set up a service account, and a service field that allows you to specify whether the service should be clusterIP, node port, or a load balancer. In Kubernetes, these fields are often set in pod templates found in controllers like deployments, jobs, and stateful sets. The values.yaml file sets the default values that helm will map to the Kubernetes objects it creates.

**Templates**

* NOTES.txt : This is the text that's going to be displayed in the terminal when somebody else downloads and installs your chart. This is the first time we've seen the helm templating syntax, which is an opening and closing set of two curly braces. All the files in the templates directory are sent through the helm template rendering engine, where the values from the first files we looked at are passed in and then converted to familiar Kubernetes objects.

* service.yml: we can see a familiar Kubernetes service manifest, but some of the values are dynamically created with helm templates
